In [16]:
import pandas as pd
import numpy as np

# 1. Load all files (same as before)
months = ['202411', '202412', '202501', '202502', '202503', '202504']
bal_files = {m: f"Account_Balance_{m}.csv" for m in months}

balances = []
for m, f in bal_files.items():
    df = pd.read_csv(f)
    df['Month'] = m
    df = df.rename(columns=lambda x: x.strip())
    acct_col = 'Account_ID' if 'Account_ID' in df.columns else [col for col in df.columns if 'Account' in col][0]
    bal_col = 'Balance' if 'Balance' in df.columns else [col for col in df.columns if 'Bal' in col][0]
    df = df.rename(columns={acct_col: 'Account_ID', bal_col: 'Balance'})
    balances.append(df[['Account_ID', 'Balance', 'Month']])

balances = pd.concat(balances, ignore_index=True)

mapping = pd.read_csv('Client_Mapping.csv').rename(columns=lambda x: x.strip())
kyc = pd.read_csv('KYC.csv').rename(columns=lambda x: x.strip())

balances = balances.merge(mapping, on='Account_ID', how='left')
balances = balances.merge(kyc, on='Account_ID', how='left')
balances['Balance'] = pd.to_numeric(balances['Balance'], errors='coerce')
balances['Month'] = balances['Month'].astype(str)

# 2. Account-level KPIs
grp = balances[balances['Balance'].notnull()].groupby('Account_ID')

first_active_month = grp['Month'].min()
last_active_month = grp['Month'].max()
months_active = grp.size()
first_balance = grp.apply(lambda x: x.loc[x['Month'] == x['Month'].min(), 'Balance'].values[0])
last_balance = grp.apply(lambda x: x.loc[x['Month'] == x['Month'].max(), 'Balance'].values[0])
min_balance = grp['Balance'].min()
max_balance = grp['Balance'].max()
avg_balance = grp['Balance'].mean()

churn_flag = last_active_month != '202504'
closure_month = last_active_month.where(churn_flag)

# Balance before closure
def balance_before_closure_func(account):
    df = account.sort_values('Month')
    if len(df) < 2:
        return np.nan
    return df.iloc[-2]['Balance']

balance_before_closure = grp.apply(balance_before_closure_func).where(churn_flag)

pct_change_first_to_last = (last_balance - first_balance) / first_balance.replace(0, np.nan)

# Was the account ever inactive (had a NaN/0 between first and last active)?
def ever_inactive_func(account):
    df = account.sort_values('Month')
    all_months = df['Month'].tolist()
    b = df['Balance'].tolist()
    # in the period from first to last, did we ever have a missing/zero?
    return int(any([pd.isna(v) or v == 0 for v in b[1:-1]])) if len(b) > 2 else 0

ever_inactive = grp.apply(ever_inactive_func)

# Add back KYC columns for each account
account_info = balances.groupby('Account_ID').agg({
    'Client_ID': 'first',
    'Client_Segment': 'first',
    'ClientLanguage': 'first',
    'BirthDate': 'first'
})

account_kpis = pd.DataFrame({
    'Account_ID': first_active_month.index,
    'Client_ID': account_info['Client_ID'],
    'Client_Segment': account_info['Client_Segment'],
    'ClientLanguage': account_info['ClientLanguage'],
    'BirthDate': account_info['BirthDate'],
    'first_active_month': first_active_month,
    'last_active_month': last_active_month,
    'months_active': months_active,
    'churn_flag': churn_flag.astype(int),
    'closure_month': closure_month,
    'balance_before_closure': balance_before_closure,
    'first_balance': first_balance,
    'last_balance': last_balance,
    'pct_change_first_to_last': pct_change_first_to_last,
    'min_balance': min_balance,
    'max_balance': max_balance,
    'avg_balance': avg_balance,
    'ever_inactive': ever_inactive
}).reset_index(drop=True)

# 3. Client-level KPIs
# Accounts at start = accounts with a non-null balance in the first month
# Accounts at end = accounts with non-null balance in the last month
accounts_start = balances[balances['Month'] == months[0]].groupby('Client_ID')['Account_ID'].nunique()
accounts_end = balances[balances['Month'] == months[-1]].groupby('Client_ID')['Account_ID'].nunique()

# Number of accounts closed for each client (sum of churn_flag)
closed_accounts = account_kpis.groupby('Client_ID')['churn_flag'].sum()
total_accounts = account_kpis.groupby('Client_ID')['Account_ID'].nunique()

# Client-level total balance (each month)
client_monthly_bal = balances.groupby(['Client_ID', 'Month'])['Balance'].sum().unstack(fill_value=0)

# Client-level balance growth and volatility
client_first_balance = client_monthly_bal[months[0]]
client_last_balance = client_monthly_bal[months[-1]]
client_balance_growth = (client_last_balance - client_first_balance) / client_first_balance.replace(0, np.nan)
client_balance_volatility = client_monthly_bal.std(axis=1)

# Combine client-level KPIs
client_kpis = pd.DataFrame({
    'Client_ID': client_monthly_bal.index,
    'accounts_at_start': accounts_start,
    'accounts_at_end': accounts_end,
    'accounts_closed': closed_accounts,
    'total_accounts': total_accounts,
    'client_balance_growth': client_balance_growth,
    'client_balance_volatility': client_balance_volatility
}).fillna(0).reset_index(drop=True)

# Merge in the main language/segment/birthdate for each client
client_firsts = account_kpis.groupby('Client_ID').agg({
    'Client_Segment': 'first',
    'ClientLanguage': 'first',
    'BirthDate': 'first'
}).reset_index()
client_kpis = client_kpis.merge(client_firsts, on='Client_ID', how='left')

# Save both tables if you want
account_kpis.to_csv("account_kpis.csv", index=False)
client_kpis.to_csv("client_kpis.csv", index=False)

print("Sample Account-level KPIs:\n", account_kpis.head())
print("\nSample Client-level KPIs:\n", client_kpis.head())

/tmp/ipython-input-16-950973098.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_balance = grp.apply(lambda x: x.loc[x['Month'] == x['Month'].min(), 'Balance'].values[0])
/tmp/ipython-input-16-950973098.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_balance = grp.apply(lambda x: x.loc[x['Month'] == x['Month'].max(), 'Balance'].values[0])
/tmp/ipython-input-16-950973098.py:50: Deprecatio

Sample Account-level KPIs:
             Account_ID           Client_ID Client_Segment ClientLanguage  \
0  AccountID0000000011  ClientID0000677488         Trader        English   
1  AccountID0000000056  ClientID0000324811       Explorer        English   
2  AccountID0000000098  ClientID0000647333         Trader        English   
3  AccountID0000000204  ClientID0000165070       Explorer        English   
4  AccountID0000000240  ClientID0000036019          Early        English   

         BirthDate first_active_month last_active_month  months_active  \
0  1938-04-26 0:00             202411            202504              6   
1  1975-07-11 0:00             202411            202504              6   
2  1960-05-21 0:00             202411            202504              6   
3  1940-08-30 0:00             202411            202504              6   
4  1968-04-01 0:00             202411            202504              6   

   churn_flag closure_month  balance_before_closure  first_balance  \


In [6]:
account_kpis

,Account_ID,Client_ID,Client_Segment,ClientLanguage,BirthDate,first_active_month,last_active_month,months_active,churn_flag,closure_month,balance_before_closure,first_balance,last_balance,pct_change_first_to_last,min_balance,max_balance,avg_balance,ever_inactive
0,AccountID0000000011,ClientID0000677488,Trader,English,1938-04-26 0:00,202411,202504,6,0,NaN,NaN,16350.12,11331.88,-0.306924,11331.88,29679.54,19242.183333,0
1,AccountID0000000056,ClientID0000324811,Explorer,English,1975-07-11 0:00,202411,202504,6,0,NaN,NaN,181.44,82.04,-0.547840,82.04,369.60,204.886667,0
2,AccountID0000000098,ClientID0000647333,Trader,English,1960-05-21 0:00,202411,202504,6,0,NaN,NaN,65414.88,44122.68,-0.325495,44122.68,119881.74,77075.370000,0
3,AccountID0000000204,ClientID0000165070,Explorer,English,1940-08-30 0:00,202411,202504,6,0,NaN,NaN,2038.68,1298.92,-0.362862,1298.92,3618.78,2275.363333,0
4,AccountID0000000240,ClientID0000036019,Early,English,1968-04-01 0:00,202411,202504,6,0,NaN,NaN,6181.56,36442.28,4.895321,6181.56,45646.26,26222.023333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50782,AccountID0001410887,ClientID0000695597,Early,English,1954-03-29 0:00,202411,202504,6,0,NaN,NaN,16269.84,13104.84,-0.194532,13104.84,32515.56,21175.853333,0
50783,AccountID0001410937,ClientID0000158264,Early,English,1999-10-22 0:00,202411,202412,2,1,202412,495.72,495.72,203.84,-0.588800,203.84,495.72,349.780000,0
50784,AccountID0001410966,ClientID0000656077,Trader,English,1937-08-06 0:00,202411,202504,6,0,NaN,NaN,61179.84,44191.56,-0.277678,44191.56,113036.88,73831.226667,0
50785,AccountID0001411009,ClientID0000302470,Early,English,1995-07-25 0:00,202411,202504,6,0,NaN,NaN,1738.44,909.16,-0.477025,909.16,3218.16,1972.826667,0


In [18]:
account_kpis.to_csv("account_kpis.csv", index=False)
client_kpis.to_csv("client_kpis.csv", index=False)

In [11]:
client_kpis

,Client_ID,accounts_at_start,accounts_at_end,accounts_closed,total_accounts,client_balance_growth,client_balance_volatility,Client_Segment,ClientLanguage,BirthDate
0,ClientID0000000011,1.0,1.0,0,1,-0.298962,8508.901922,Trader,English,1959-01-20 0:00
1,ClientID0000000039,1.0,1.0,0,1,-0.640593,2060.386426,Explorer,English,1960-01-13 0:00
2,ClientID0000000052,1.0,1.0,0,1,-0.335647,20533.740323,Explorer,English,1959-05-20 0:00
3,ClientID0000000054,1.0,1.0,0,1,-0.338571,69516.418699,Explorer,English,1952-07-18 0:00
4,ClientID0000000057,1.0,0.0,1,1,-1.000000,8.946601,Trader,English,1992-05-29 0:00
...,...,...,...,...,...,...,...,...,...,...
50502,ClientID0000733394,1.0,1.0,0,1,-0.629677,175.341385,Explorer,English,1949-09-21 0:00
50503,ClientID0000733414,1.0,1.0,0,1,-0.332497,102.183524,Explorer,English,1997-07-27 0:00
50504,ClientID0000733416,1.0,1.0,0,1,-0.336166,76438.954022,Explorer,English,1972-03-14 0:00
50505,ClientID0000733449,1.0,1.0,0,1,-0.220264,8307.761991,Explorer,English,1990-08-01 0:00


In [17]:
display(account_kpis[account_kpis['churn_flag'] == 1])

,Account_ID,Client_ID,Client_Segment,ClientLanguage,BirthDate,first_active_month,last_active_month,months_active,churn_flag,closure_month,balance_before_closure,first_balance,last_balance,pct_change_first_to_last,min_balance,max_balance,avg_balance,ever_inactive
25,AccountID0000000828,ClientID0000689454,Trader,English,1948-07-29 0:00,202411,202502,4,1,202502,66000.00,36000.00,48000.00,0.333333,36000.00,66000.00,50500.000000,0
74,AccountID0000002121,ClientID0000684530,Trader,English,1975-12-21 0:00,202411,202503,5,1,202503,53.76,40.32,35.84,-0.111111,35.84,73.92,52.416000,0
86,AccountID0000002477,ClientID0000658116,Trader,English,1950-09-30 0:00,202411,202501,3,1,202501,-0.52,-0.36,-0.66,0.833333,-0.66,-0.36,-0.513333,0
104,AccountID0000003012,ClientID0000491852,Trader,English,1980-10-11 0:00,202411,202411,1,1,202411,NaN,417.24,417.24,0.000000,417.24,417.24,417.240000,0
117,AccountID0000003524,ClientID0000707622,Explorer,English,1957-02-25 0:00,202411,202501,3,1,202501,-1.04,-0.72,-1.32,0.833333,-1.32,-0.72,-1.026667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50462,AccountID0001401802,ClientID0000145648,Trader,English,1995-12-30 0:00,202411,202412,2,1,202412,406.44,406.44,574.08,0.412459,406.44,574.08,490.260000,0
50467,AccountID0001401969,ClientID0000102208,Trader,English,1978-02-25 0:00,202411,202503,5,1,202503,0.96,0.72,0.64,-0.111111,0.64,1.32,0.936000,0
50605,AccountID0001406082,ClientID0000659925,Explorer,English,1998-09-02 0:00,202411,202502,4,1,202502,111.54,2349.00,81.12,-0.965466,81.12,3445.52,1496.795000,0
50634,AccountID0001406734,ClientID0000526679,Trader,English,2001-08-05 0:00,202411,202412,2,1,202412,203.40,203.40,327.60,0.610619,203.40,327.60,265.500000,0
